In [1]:
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('../feature/data_cutted.csv')

In [3]:
data.head()

,id,content,source
0,yhM4ivIm,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train
1,OeJidjGX,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train
2,4wWp3dGK,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train
3,T0senQto,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train
4,1jAboq7C,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['id'] = le.fit_transform(data['id'])

#list(le.inverse_transform([2, 2, 1]))

In [5]:
data.head()

,id,content,source
0,98495,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train
1,40279,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train
2,8195,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train
3,47349,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train
4,2839,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train


In [6]:
def get_words(txt):
    c = Counter()  
    for x in txt.split( ):  
        if len(x)>1 and x != '\r\n':  
            c[x] += 1
    return c

In [7]:
data['vector_content'] = data['content'].apply(lambda x: get_words(str(x)))

In [8]:
data.head()

,id,content,source,vector_content
0,98495,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train,"{'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ..."
1,40279,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train,"{'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1..."
2,8195,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train,"{'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':..."
3,47349,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train,"{'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'..."
4,2839,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train,"{'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ..."


In [9]:
def get_cosine_new(vec1, vec2):
    try:
        present_count = 0
        total_count = 0
        for word,count in vec1.items():
            total_count += count
            if word in vec2:
                present_count += count
        return present_count/max(total_count,1)
    except:
        return 0.0

In [10]:
def find_relation(inp_df):
    relation_df = []
    train = inp_df.loc[data['source']=='train'].reset_index()
    test = inp_df.loc[data['source']=='test'].reset_index()
    
    for i in range(len(test)):
        #print (i)
        for j in range(len(train)):
            rel_df = dict()
            try:
                rel_df["cosine_abstract"] = get_cosine_new(test['vector_content'][i],train['vector_content'][j])
            except:
                rel_df["cosine_abstract"] = 0.0
            
            #print (rel_df)
            if rel_df["cosine_abstract"] > 0.0:
                rel_df["id"] = test["id"][i]
                rel_df["related_id"] = train["id"][j]
                relation_df.append(rel_df)
            else:
                pass
    final_df = pd.DataFrame(relation_df)
    return final_df

In [20]:
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

train.drop(['source','content'],axis=1,inplace=True)
test.drop(['source'],axis=1,inplace=True)

train.rename(columns = {'id':'related_id','vector_content':'train_vector'},inplace=True)
test.rename(columns = {'content':'test_content'},inplace=True)

C:\Users\kuhung\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\kuhung\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
F:\software\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [21]:
from tqdm import *
import numpy as np

In [40]:
def get_compare_df(train,row_id,row_content):
    temp = train.copy()
    temp['id'] = row_id
    temp['test_content'] = row_content
    temp['test_vector'] = temp['test_content'].apply(lambda x: get_words(str(x)))
    temp.drop('test_content',axis=1,inplace=True)
    temp['cosine_abstract'] = temp.apply(lambda row:get_cosine_new(row['train_vector'],row['test_vector']),axis=1)     

    print(temp.head())
    return 

In [41]:
for index, row in tqdm(test.head(5).iterrows()):
    compare_df_one = get_compare_df(train,row['id'],row['test_content'])

0it [00:00, ?it/s]

   related_id                                       train_vector     id  \
0       98495  {'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ...  59765   
1       40279  {'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1...  59765   
2        8195  {'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':...  59765   
3       47349  {'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'...  59765   
4        2839  {'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ...  59765   

                                         test_vector  cosine_abstract  
0  {'文中': 1, '介绍': 1, '工程': 1, '施工': 1, '管理': 1, ...              0.0  
1  {'文中': 1, '介绍': 1, '工程': 1, '施工': 1, '管理': 1, ...              0.0  
2  {'文中': 1, '介绍': 1, '工程': 1, '施工': 1, '管理': 1, ...              0.0  
3  {'文中': 1, '介绍': 1, '工程': 1, '施工': 1, '管理': 1, ...              0.0  
4  {'文中': 1, '介绍': 1, '工程': 1, '施工': 1, '管理': 1, ...              0.0  


1it [00:06,  6.26s/it]

   related_id                                       train_vector     id  \
0       98495  {'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ...  82902   
1       40279  {'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1...  82902   
2        8195  {'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':...  82902   
3       47349  {'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'...  82902   
4        2839  {'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ...  82902   

                                         test_vector  cosine_abstract  
0  {'文章': 1, '简要': 1, '介绍': 1, '目前': 1, '国内': 1, ...              0.0  
1  {'文章': 1, '简要': 1, '介绍': 1, '目前': 1, '国内': 1, ...              0.0  
2  {'文章': 1, '简要': 1, '介绍': 1, '目前': 1, '国内': 1, ...              0.0  
3  {'文章': 1, '简要': 1, '介绍': 1, '目前': 1, '国内': 1, ...              0.0  
4  {'文章': 1, '简要': 1, '介绍': 1, '目前': 1, '国内': 1, ...              0.0  


2it [00:14,  7.37s/it]

...

3it [00:20,  6.75s/it]

   related_id                                       train_vector     id  \
0       98495  {'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ...  67034   
1       40279  {'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1...  67034   
2        8195  {'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':...  67034   
3       47349  {'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'...  67034   
4        2839  {'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ...  67034   

                                         test_vector  cosine_abstract  
0  {'阴离子': 3, '黏土': 3, '水中': 1, 'as': 3, '吸附': 6,...              0.0  
1  {'阴离子': 3, '黏土': 3, '水中': 1, 'as': 3, '吸附': 6,...              0.0  
2  {'阴离子': 3, '黏土': 3, '水中': 1, 'as': 3, '吸附': 6,...              0.0  
3  {'阴离子': 3, '黏土': 3, '水中': 1, 'as': 3, '吸附': 6,...              0.0  
4  {'阴离子': 3, '黏土': 3, '水中': 1, 'as': 3, '吸附': 6,...              0.0  


4it [00:29,  7.36s/it]

   related_id                                       train_vector     id  \
0       98495  {'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ...  43133   
1       40279  {'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1...  43133   
2        8195  {'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':...  43133   
3       47349  {'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'...  43133   
4        2839  {'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ...  43133   

                                         test_vector  cosine_abstract  
0  {'包含': 1, '超平面': 2, '组成': 2, '集合': 2, '信源': 1,...              0.0  
1  {'包含': 1, '超平面': 2, '组成': 2, '集合': 2, '信源': 1,...              0.0  
2  {'包含': 1, '超平面': 2, '组成': 2, '集合': 2, '信源': 1,...              0.0  
3  {'包含': 1, '超平面': 2, '组成': 2, '集合': 2, '信源': 1,...              0.0  
4  {'包含': 1, '超平面': 2, '组成': 2, '集合': 2, '信源': 1,...              0.0  


5it [00:35,  7.11s/it]


In [ ]:
%%time
frequency_vec = find_relation(data)

In [ ]:
frequency_vec['id'] = le.inverse_transform(frequency_vec['id'])
frequency_vec['related_id'] = le.inverse_transform(frequency_vec['related_id'])

In [ ]:
frequency_vec.to_csv('../feature/cosine_abstract.csv',index=False)

In [ ]:
frequency_vec = pd.read_csv('../feature/cosine_abstract.csv')

In [ ]:
print(frequency_vec.shape)

In [ ]:
print(frequency_vec.describe())

In [ ]:
#frequency_pivot.replace({'﻿mhY5opF':'?mhY5opF4'},inplace=True) 
frequency_pivot = frequency_vec.set_index(['id','related_id']).unstack()

In [ ]:
result = frequency_pivot.idxmax(axis = 1).apply(lambda x : str(x)[21:29])

In [ ]:
import numpy as np
result_pro = np.max(frequency_pivot,axis=1)

In [ ]:
frequency_pivot.index

In [ ]:
sub = pd.DataFrame({'test_id':frequency_pivot.index,'result':result,'pro':result_pro})

In [ ]:
sub.replace({'?mhY5opF':'?mhY5opF4'},inplace=True) 

In [ ]:
sub.describe()

In [ ]:
def threshold(result,distance):
    if distance > 0.545455:
        return result
    else:
        return 

In [ ]:
sub['result'] = sub.apply(lambda row: threshold(row['result'],row['pro']),axis=1)

In [ ]:
sub.drop('pro',axis=1).to_csv('../output/frequency_threshold50.csv',index=False)

In [ ]:
sub.head()